In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [3]:
import sys
import os
sys.path.append('/content/drive/MyDrive/ColabNotebooks/DL/ViT')
os.chdir("/content/drive/MyDrive/ColabNotebooks/DL/ViT")

In [3]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [10]:
import torch
import matplotlib.pyplot as plt
from data_util import get_data_from_kaggle, restructure_date_information, get_static_df, one_label_scale_static_df, scale_stock_data
from constant import Constant
import pandas as pd
from torch.utils.data import DataLoader ,random_split
from data_util import get_feature_length
import glob
import data
from data import ViT_Dataset
import sklearn
from transformers import ViTConfig, ViTModel
import torch.nn as nn
import torch.nn.functional as F
from ViT import ViT_Model

In [9]:
pip install einops 

   ---------------------------------------- 0.0/43.2 kB ? eta -:--:--
   ---------------------------- ----------- 30.7/43.2 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 43.2/43.2 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

cuda


In [12]:
constant = Constant() # initialize variables we will use



---

\

Preprocesssing

\

---

In [13]:
market = 'sp500' # [forbes2000, nasdaq, nyse, sp500]
year = '2017'

In [14]:
# stock = get_data_from_kaggle(market = 'sp500', start_date = f'01-01-{year}')

In [15]:
# generating static df can take a while because there is a rate limit with yfinance api
# in my case I don't need statick
# static = get_static_df(stock,  constant.static_variables)
# stock.to_csv(f'dataset/sp500_{year}.csv', index=False)
stock = pd.read_csv(f'dataset/sp500_{year}.csv')

In [16]:
# static = get_static_df(stock,  constant.static_variables)

In [17]:
stock = stock.dropna()

In [18]:
# note: stock name and order might not be the same as the others as the names are out of order
# check with team after

In [19]:
stock_scaled, stock_scalar = scale_stock_data(stock, constant.columns_to_scale)
stock_scaled

,Date,Low,Open,Volume,High,Close,Adjusted Close,Stock Name
0,2017-01-03,0.000000,0.000000,0.108231,0.000000,0.000000,0.000000,A
1,2017-01-04,0.008217,0.007499,0.113930,0.004743,0.004594,0.004368,A
2,2017-01-05,0.004717,0.008399,0.091776,0.002409,0.000377,0.000358,A
3,2017-01-06,0.006239,0.005249,0.188017,0.009938,0.011296,0.010740,A
4,2017-01-09,0.016511,0.015598,0.166526,0.013627,0.012426,0.011814,A
...,...,...,...,...,...,...,...,...
605210,2022-12-06,0.518709,0.522872,0.132240,0.522823,0.516862,0.526255,ZTS
605211,2022-12-07,0.504664,0.516396,0.170338,0.514121,0.502468,0.511865,ZTS
605212,2022-12-08,0.503731,0.503906,0.156301,0.516971,0.520101,0.529492,ZTS
605213,2022-12-09,0.522077,0.521433,0.236376,0.527047,0.518610,0.528002,ZTS


In [20]:
processed_df = restructure_date_information(stock_scaled)

In [21]:
processed_df.head()

,Low,Open,Volume,High,Close,Adjusted Close,Stock Name,Month,Day,Day of The Week,Week of The Year
0,0.000000,0.000000,0.108231,0.000000,0.000000,0.000000,A,0,2,1,0
1,0.008217,0.007499,0.113930,0.004743,0.004594,0.004368,A,0,3,2,0
2,0.004717,0.008399,0.091776,0.002409,0.000377,0.000358,A,0,4,3,0
3,0.006239,0.005249,0.188017,0.009938,0.011296,0.010740,A,0,5,4,0
4,0.016511,0.015598,0.166526,0.013627,0.012426,0.011814,A,0,8,0,1


In [22]:
# static now

In [23]:
# static.to_csv(f'dataset/sp500_{year}_static.csv')
static = pd.read_csv(f"dataset/sp500_{year}_static.csv", index_col=0)
static_df = one_label_scale_static_df(static,constant.static_variables)



---

\

---



---



In [24]:
history_cat_feature_num_list, history_cont_feature_num = get_feature_length(processed_df, constant.feature_variables)
static_cat_feature_num_list , static_cont_feature_num  = get_feature_length(static, constant.static_variables)
future_cat_feature_num_list , _                        = get_feature_length(processed_df, constant.future_feature)
_                           , prediction_con           = get_feature_length(processed_df, constant.prediction_feature)

In [25]:
history_cat_feature_num_list, history_cont_feature_num

([12, 31, 5, 53], 4)

In [26]:
history_length = 90
prediction_length = 15

In [27]:
# use this if want to reset ViT Dataset
# del stock_ViT_dataset

In [28]:
stock_ViT_dataset = ViT_Dataset(stock_df= processed_df ,
                        static_df=static_df,
                        constant_variable=constant,
                        history_length= history_length,
                        prediction_length= prediction_length,
                        device = device)



---

Training



---



In [29]:
batch_size = 128

In [30]:
train_size = int(0.8 * stock_ViT_dataset.__len__())
test_size = len(stock_ViT_dataset) - train_size

In [31]:
train_dataset, test_dataset = random_split(stock_ViT_dataset, [train_size, test_size])

In [32]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,shuffle=True)

In [ ]:
count =0
for static_cont_input, static_cat_input,history_cont_input, history_cat_input, future_input, prediction, image_tensor, prediction_classification in train_loader:
    break
print(image_tensor.device)

cuda:0


In [34]:
image_tensor.size()

torch.Size([128, 1, 48, 48])

In [ ]:
# training

import torch.optim as optim
# Call the model with the patches_tensor
model = ViT_Model(image_size = 48, patch_size=8, embedding_dim=256, num_heads=8, num_classes=2, num_layers=6, mlp_dim = 512, embedding_dropout = 0.1 )
model.to(device)
# loss function
loss_fn = nn.BCELoss().to(device)  # binary cross-entropy loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # optimizer


In [ ]:
model.train()
num_epochs = 10
# training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:

      # this is where I will use patches_tensor and prediction_classification from each batch
      static_cont_input, static_cat_input, history_cont_input, history_cat_input, future_input, prediction, patches_tensor, prediction_classification = batch
      # forward pass
      outputs = model(patches_tensor)  # (batch_size, 1)
      loss = loss_fn(outputs, prediction_classification) # loss

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      running_loss += loss.item()

    # avg epoch loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")



---
Prediction


---




In [ ]:
# model prediction

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    # Get a sample from the dataset
    sample = ViT_Dataset[0]  # Assuming this gets one sample

    # Extract the input patches
    patches_tensor = sample[6].unsqueeze(0)  # Add batch dimension

    # Get the prediction (probability of class 1)
    prediction_prob = model(patches_tensor)

    # Convert probability to class (0 or 1)
    predicted_class = (prediction_prob > 0.5).int()

    print("Predicted Class:", predicted_class.item())  # Output: 0 or 1





---

\

Creating Model Instance

\

---

In [ ]:
# pretraining?

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch
import torch.nn.functional as F

# Simulate stock data (replace this with your actual stock data)
stock_data = np.random.rand(100)  # Example: 100 stock price points

# Step 1: Generate a plot
fig, ax = plt.subplots(figsize=(4, 4))  # Adjust figsize for desired image dimensions
ax.plot(stock_data, color='blue')  # Example: Line plot for stock data
ax.axis('off')  # Remove axes for cleaner image

# Step 2: Convert the plot to an image in-memory
fig.canvas.draw()
image = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))  # H x W x C format

plt.close(fig)  # Close the figure to free resources

# Step 3: Convert to PyTorch tensor and normalize
image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0  # C x H x W

# Step 4: Ensure dimensions are divisible by patch size
patch_size = 4
C, H, W = image_tensor.shape
H_padded = (H + patch_size - 1) // patch_size * patch_size  # Pad height to nearest multiple
W_padded = (W + patch_size - 1) // patch_size * patch_size  # Pad width to nearest multiple
image_tensor = torch.nn.functional.pad(image_tensor, (0, W_padded - W, 0, H_padded - H))

# Step 5: Split into patches
image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension
patches = F.unfold(image_tensor, kernel_size=patch_size, stride=patch_size)
patches = patches.transpose(1, 2).reshape(-1, C, patch_size, patch_size)

# Output: Patches ready for ViT
print(f"Patches shape: {patches.shape}")  # (num_patches, C, patch_size, patch_size)


: 

In [ ]:
hidden_size = 128
batch_size = 128
dropout = .2
num_head = 4
lr= 0.0001
momentum=0.9
num_epochs = 10
max_grad_norm = 1
split_ratio = 0.8

In [ ]:
model = ViT(
            device = device)
model.to(device)

ViT model extracts features